In [2]:
import pandas as pd
from plant_model import PlantUnits
from utils import get_raw_data_by_time
from datetime import datetime, timedelta
import numpy as np
from utils import get_plant_characteristics
from utils import get_demand_data
from data_validations_preprocessing import capacity_checks_of_plants,cost_checks_of_plant,filling_missing_demand_withmean,missing_demand_at_timeblock_level_and_filling
from demand_model import Demand
from statistics import mean

In [3]:
COAL_RAMP_UP_PERCENT = 0.01
COAL_RAMP_DOWN_PERCENT = 0.015
COAL_EFFICIENCY_RATE = 0.55
DEVELOPMENT_PERIOD_START_TIME = datetime(2022, 1, 1)
DEVELOPMENT_PERIOD_END_TIME = datetime(2022, 2, 1)
MODEL_PERIOD_START_TIME = datetime(2021, 2, 1)
MODEL_PERIOD_END_TIME = datetime(2021, 2, 2)
INFINITE_CAPACITY = 8000
TIME_LEVEL = ['date','hour_of_day','time_block_of_day']
DEMAND_LEVEL = ['date','hour_of_day','avg_unit_current_load']
FILE_NAME = "RawData/upsldc_plant_unit_time_block.csv"

In [4]:
raw_data = pd.read_csv("RawData/upsldc_plant_unit_time_block.csv")
raw_data['date'] = pd.to_datetime(raw_data['data_capture_time_block_start']).dt.date

In [5]:
plant_unit_timeblocks = get_raw_data_by_time(raw_data,DEVELOPMENT_PERIOD_START_TIME, DEVELOPMENT_PERIOD_END_TIME)

In [6]:
model_data = get_raw_data_by_time(raw_data,MODEL_PERIOD_START_TIME, MODEL_PERIOD_END_TIME)

In [7]:
plant_units = get_plant_characteristics(plant_unit_timeblocks)

/home/ubuntu/anaconda3/envs/uppower/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1216: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/ubuntu/anaconda3/envs/uppower/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1216: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/ubuntu/UP-Power-Scheduling/utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_data['upsldc_unit_capacity'] = INFINITE_CAPACITY


In [8]:
demand_of_UP_bydate_byhour_units,demand_UP = get_demand_data(model_data)

In [9]:
print(capacity_checks_of_plants(plant_units))
print(cost_checks_of_plant(plant_units))

capacity checks done
This plant unit has wrong capacityPARICHHA TPS unit:2


In [15]:
average_demand_dict=filling_missing_demand_withmean(demand_of_UP_bydate_byhour_units)
demand_of_UP_bydate_byhour_units_filled,demand_values = missing_demand_at_timeblock_level_and_filling(demand_of_UP_bydate_byhour_units,average_demand_dict)


10378.5484747227


/home/ubuntu/UP-Power-Scheduling/data_validations_preprocessing.py:50: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  mapping_hour_time_block = pd.read_csv("RawData/hours_timeblock_mapping.csv",header=None,index_col=0,squeeze=True).to_dict()


In [20]:
plant_names = []
plant_production_costs = {}
plant_capacity = {}
for plant in plant_units:
    if(plant.average_variable_cost>0):
        plant_names.append(plant.name)
        plant_production_costs[plant.name] = plant.average_variable_cost
        plant_capacity[plant.name] = plant.capacity
        break


scheduling_time_blocks = list(range(1,97))
scheduling_dates =  demand_UP['date'].unique()



In [28]:
from pulp import *
#creating the LP problem instance
prob = LpProblem("production_cost_minimization", LpMinimize)


#Creating the production variables
production_vars = LpVariable.dicts(name = "production_units",indices = [(i,j,k) for i in plant_names for j in scheduling_dates for k in scheduling_time_blocks],lowBound=0)

#LP Objective function 
prob += lpSum([plant_production_costs[i]*production_vars[(i,j,k)] for i in plant_names for j in scheduling_dates for k in scheduling_time_blocks]), "Sum of production costs"

for date in scheduling_dates:
        for time_block in scheduling_time_blocks:
            demand_date_timeblock = str(date) + "-"+str(time_block)
            prob+= (lpSum(production_vars[(plant,date,time_block)] for plant in plant_names) >= demand_values[demand_date_timeblock])


# capacity constraint 
# capacity of the power plant cannot be exceeded at any hour

for plant in plant_names:
    for date in scheduling_dates:
        for time_block in scheduling_time_blocks:
                prob += production_vars[(plant,date,time_block)] <= plant_capacity[plant]

In [29]:
prob.writeLP("test.lp")

[production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_1),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_10),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_11),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_12),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_13),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_14),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_15),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_16),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_17),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_18),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_19),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_2),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_20),
 production_units_('ANPARA_C_TPS_unit:1'